# bettertransfomrers 
    主要做融合和跳过算子的优化，同时，也把注意力变成flash attention
    方法在PreTrainedModel中
    optimum.BetterTransformer.transform
    不支持8bit格式转换
    遍历替换一些特定的模块层

In [ ]:
#cpu加速，这是把模型转化为更好实现的格式
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder")
model.to_bettertransformer()
#model = model.reverse_bettertransformer()

# onnx 快速实现

In [ ]:
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForQuestionAnswering

model = ORTModelForQuestionAnswering.from_pretrained("optimum/roberta-base-squad2")
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

onnx_qa = pipeline("question-answering", model=model, tokenizer=tokenizer)

question = "What's my name?"
context = "My name is Philipp and I live in Nuremberg."
pred = onnx_qa(question, context)

In [ ]:
#编译可以直接加速推断30%
import torch
from PIL import Image
import requests
import numpy as np
from transformers import AutoImageProcessor, AutoModelForImageClassification

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224").to("cuda")
model = torch.compile(model)#++++++++++++加速30%+++++++++++++

processed_input = processor(image, return_tensors='pt').to(device="cuda")

with torch.no_grad():
    _ = model(**processed_input)


In [ ]:
#help函数，测量GPU内存占用峰值
def bytes_to_giga_bytes(bytes):
    return bytes / 1024 / 1024 / 1024
bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

In [ ]:
#LLM模型高效执行，靠的是以下三个
Lower Precision
Flash Attention
Architectural Innovations

In [ ]:
Lower Precision
现在所有的LLM都是用bfloat16格式训练的from_pretrained(..., torch_dtype='bfloat16')

量化过的模型这样用
!pip install bitsandbytes
model = AutoModelForCausalLM.from_pretrained("bigcode/octocoder", 
                                             load_in_8bit=True, 
                                             pad_token_id=0)

In [ ]:
Flash Attention1
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    #pipe就是原来的模型制作的，只要这样用就能加快执行
    #不用pipe也行，自己写model(**inputs)
    result = pipe(long_prompt, max_new_tokens=60)[0]["generated_text"][len(long_prompt):]

In [ ]:
Architectural Innovations

RoPE
    
key-value cache ：聊天任务就是靠着这个实现的
generation_output = model.generate(**model_inputs, 
                                   max_new_tokens=60, 
                                   return_dict_in_generate=True,
                                   do_sample=True#采样生成，结果多变
                )
decoded_output = tokenizer.batch_decode(generation_output.sequences)[0]

prompt = decoded_output + "\nQuestion: How can I modify the function above to return Mega bytes instead?\n\nAnswer: Here"
model_inputs = tokenizer(prompt, return_tensors='pt')
generation_output = model.generate(
  **model_inputs,
  past_key_values=generation_output.past_key_values,
  max_new_tokens=60,
  return_dict_in_generate=True
)
tokenizer.batch_decode(generation_output.sequences)[0][len(prompt):]

# 量化工作

In [ ]:
!pip install -U -q git+https://github.com/huggingface/transformers.git
!pip install -U -q quanto
!pip install -U -q accelerate
!pip install -q datasets

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
import torch

model_id = "bigscience/bloom-560m"
device_map = {'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.26': 0,
 'model.layers.27': 'cpu',
 'model.layers.28': 'cpu',
 'model.layers.31': 'disk',
 'model.norm': 'cpu',
 'lm_head':0}
quantization_config = QuantoConfig(weights="int8")
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             torch_dtype=torch.float32,
                                             quantization_config=quantization_config,
                                             device_map=device_map)

#把参数传入就完成量化

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [1]:
text = "Hello my name is"
device = "cuda"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

NameError: name 'tokenizer' is not defined